In [1]:
import geopandas as gpd
import fiona
import pandas as pd
import matplotlib.pyplot as plt
import alphashape
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon, MultiPoint
from geopy.distance import geodesic
import re 
import os
import requests

In [4]:
#Retrieve Links 
df = pd.read_csv("Parivesh_form_A_pi_All.csv", low_memory=False)
links_df = df.iloc[:, 50:54]
pattern = r'forestsclearance\.nic\.in/\.\./DownloadPdfFile\.aspx\?FileName=[^\'"]+'

links_df["Maps of forest land proposed to be diverted  :Segment wise details"] = links_df["Maps of forest land proposed to be diverted  :Segment wise details"].astype(str)

all_matches = []

# Iterate over each row in the DataFrame
for index, row in links_df.iterrows():
    # Extract links from the specified column using regex pattern
    matches = re.findall(pattern, row["Maps of forest land proposed to be diverted  :Segment wise details"])
    # Append the matches for the current row to the list
    all_matches.append(matches)

# Create a new DataFrame with the matches
matches_df = pd.DataFrame({'Matches': all_matches})

matches_df['Proposal Information'] = df['General Details : Project Details  :(i).      Proposal No. : ']
matches_df = matches_df[matches_df['Matches'].apply(len) > 0]

In [6]:
#Coordinate Retrieval Function 

def download_link_find_distance(url):
    # Add "https://" prefix if missing
    if not url.startswith('https://'):
        url = 'https://' + url

    local_filename = 'downloaded_file.kml'

    centroids = []
    max_latitudes = []
    min_latitudes = []
    max_longitudes = []
    min_longitudes = []

    try:
        # Send a GET request to the URL
        response = requests.get(url, verify=False)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Open a file and write the content of the response to it
            with open(local_filename, 'wb') as f:
                f.write(response.content)
        else:
            return

        # Reading KML file into GeoDataFrame
        gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
        geo_data_frame = gpd.read_file(local_filename, driver='KML')

        # Iterate through each row in the GeoDataFrame
        for index, row in geo_data_frame.iterrows():
            # Extract the geometry object
            geometry = row['geometry']

            # Determine the geometry type
            geom_type = geometry.geom_type

            # If it is a POLYGON, calculate the CENTROID 
            if geom_type == 'Polygon':
                polygon = geometry
                # Calculate centroid
                centroid = polygon.centroid
                centroids.append(centroid)
        

            # If it is a LINESTRING, provide the MIN lat lon and MAX lat lon 
            if geom_type == 'LineString' or geom_type == 'Polygon':
                linestring = geometry
                # Calculate min/max latitudes and longitudes
                min_lon, min_lat, max_lon, max_lat = linestring.bounds
                min_latitudes.append(min_lat)
                max_latitudes.append(max_lat)
                min_longitudes.append(min_lon)
                max_longitudes.append(max_lon)
            
    except Exception as e:
        print("An error occurred:", e)

    finally:
        # Delete the file after processing
        if os.path.exists(local_filename):
            os.remove(local_filename)

    # Return the collected data
    return centroids, min_latitudes, max_latitudes, min_longitudes, max_longitudes

In [ ]:
#Running Function and aggregating them to the table 

# Initialize lists to hold the aggregated results
all_centroids = []
all_min_latitudes = []
all_max_latitudes = []
all_min_longitudes = []
all_max_longitudes = []

# Iterate over each row in the DataFrame
for index, row in matches_df.iterrows():
    # Initialize lists to hold the results for this row
    row_centroids = []
    row_min_latitudes = []
    row_max_latitudes = []
    row_min_longitudes = []
    row_max_longitudes = []

    # Iterate over each URL in the row
    for url in row['Matches']:
        # Apply the download_link_find_distance function
        centroids, min_latitudes, max_latitudes, min_longitudes, max_longitudes = download_link_find_distance(url)
        
        # Extend the row lists with the results for this URL
        row_centroids.extend(centroids)
        row_min_latitudes.extend(min_latitudes)
        row_max_latitudes.extend(max_latitudes)
        row_min_longitudes.extend(min_longitudes)
        row_max_longitudes.extend(max_longitudes)

    # Append the row lists to the aggregated lists
    all_centroids.append(row_centroids)
    all_min_latitudes.append(row_min_latitudes)
    all_max_latitudes.append(row_max_latitudes)
    all_min_longitudes.append(row_min_longitudes)
    all_max_longitudes.append(row_max_longitudes)

# Convert the lists to DataFrame columns
matches_test['All_Centroids'] = all_centroids
matches_test['All_Min_Latitudes'] = all_min_latitudes
matches_test['All_Max_Latitudes'] = all_max_latitudes
matches_test['All_Min_Longitudes'] = all_min_longitudes
matches_test['All_Max_Longitudes'] = all_max_longitudes


/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


An error occurred: 'NoneType' object has no attribute 'drvsupport'


/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


An error occurred: 'NoneType' object has no attribute 'drvsupport'


/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


An error occurred: 'NoneType' object has no attribute 'drvsupport'


/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


An error occurred: 'NoneType' object has no attribute 'drvsupport'


/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forestsclearance.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
